In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import lxml
import wget
import zipfile

In [2]:
# Set up splinter because "view source" doesn't match "inspect"

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been 

In [3]:
# URL of page to be scraped
url = "https://s3.amazonaws.com/tripdata/index.html"

# Visit the browser
browser.visit(url)

# Set up BeautifulSoup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Find the tag containing the desired information
results = soup.find_all('a', class_='')
results[0]

IndexError: list index out of range

In [5]:
# Create list of years, and names for the subsequent lists
years_list = []
names_list = []

for i in range(0, (len(results) - 1 )): # the "-1" will filter out index.html
    
    # extract the date
    date = results[i]["href"].split('/')[-1].split('-')[0]
    
    # filter out Jersey City (JC) data
    if (date != "JC"):
        
        # Extract the year
        year = int(date[0:4])
                
        if (year not in years_list):
            
            # Add the found year to the years_list
            years_list.append(year)
            
            # Create name, append to list
            name = f"data_{year}"
            names_list.append(name)

In [6]:
for name in names_list:
    globals()[f"{name}"] = []

In [7]:
# Loop through every link in the list
for i in range(0, (len(results) - 1 )): # the "-1" will filter out index.html
    
    # Find the date
    date = results[i]["href"].split('/')[-1].split('-')[0]
    
    # filter out Jersey City (JC) data
    if (date != "JC"):
        
        # Find the year
        year = int(date[0:4])

        # Loop through every list
        for j in range (0, len(names_list)):

            if (year == years_list[j]):
                
                # Append any applicable link to the list
                globals()[f"{names_list[j]}"].append(results[i]["href"])

In [9]:
year_selection = input(f"Choose a year between {years_list[0]} and {years_list[-1]}:")

IndexError: list index out of range

In [10]:
years_list[0]

IndexError: list index out of range

In [10]:
selected_data = globals()[f"data_{year_selection}"]
limit = len(selected_data)

In [11]:
# Create lists of zip names and csv names to use when calling the file
zip_names = []
csv_names = []

# for entry in results:
for i in range(0, limit):
    
    # Get the link
    link = selected_data[i]

    # Use the link to get the zip and csv names, in case the .text is mismatched
    zip_name = link.split('/')[-1]
    zip_names.append(zip_name)

    csv_name = zip_name.split('.')[0] + ".csv"
    csv_names.append(csv_name)
   
    # Download the zip
    wget.download(link, out = "zipfiles") 


100% [....................................................] 66434482 / 66434482

In [22]:
#Save as CSVs

for i in range(0, limit):
    
    # find and read the zip file
    find_zip = "zipfiles/" + zip_names[i]
    zf = zipfile.ZipFile(find_zip)
    df = pd.read_csv(zf.open(csv_names[i]))

    #create the csv
    df.to_csv("data/" + csv_names[i], index = False)

C:\Users\emera\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [21]:
df
station_df = pd.DataFrame(columns = ['station_name', 'station_id', 'latitude', 'longitude'])

station_names = []

for row in df:
    if row[start_station_name] not in station_names:
        start_station_name = row["start_station_name"] 
        start_station_id = row["start_station_id"] 
        start_lat = row["start_lat"] 
        start_lng = row["start_lng"] 
    
        station_df = station_df.append({'station_name': start_station_name, 'station_id': start_station_id, 'latitude': start_lat, 'longitude': start_lng}, ignore_index = True)
    
        station_names.append(start_station_name) 
    
    
    



,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,FA2F660C8D433037,electric_bike,2022-03-04 18:58:36,2022-03-04 19:13:51,Washington St & Barrow St,5847.08,University Pl & E 14 St,5905.14,40.731911,-74.008769,40.734814,-73.992085,member
1,03B6FE6FAEE61465,classic_bike,2022-03-06 20:37:09,2022-03-06 20:45:28,Washington St & Barrow St,5847.08,W 11 St & 6 Ave,5914.03,40.731911,-74.008769,40.735324,-73.998004,member
2,6E6995761B4A5760,classic_bike,2022-03-01 15:13:49,2022-03-01 15:19:53,1 Ave & E 110 St,7522.02,E 110 St & Madison Ave,7587.14,40.792327,-73.938300,40.796154,-73.947821,member
3,AA51CCF6CEDD6033,classic_bike,2022-03-11 13:59:03,2022-03-11 14:06:43,Crescent St & Broadway,6827.11,31 Rd & 21 St,6929.02,40.763359,-73.928647,40.766824,-73.930486,member
4,72AA3A7D430D8EF8,classic_bike,2022-03-08 14:59:16,2022-03-08 15:05:04,Crescent St & Broadway,6827.11,31 Rd & 21 St,6929.02,40.763359,-73.928647,40.766824,-73.930486,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893439,79E49EA8727CA662,classic_bike,2022-03-30 21:12:45,2022-03-30 21:22:15,E 72 St & York Ave,6889.12,E 58 St & 3 Ave,6762.02,40.766638,-73.953483,40.760958,-73.967245,member
1893440,9C4F6A095779AAE2,electric_bike,2022-03-21 09:33:21,2022-03-21 09:39:45,W 48 St & Rockefeller Plaza,6626.11,E 58 St & 3 Ave,6762.02,40.757769,-73.979294,40.760958,-73.967245,member
1893441,C1966A89B5078124,electric_bike,2022-03-04 14:31:36,2022-03-04 14:43:16,Central Park West & W 85 St,7354.01,E 58 St & 3 Ave,6762.02,40.784760,-73.969862,40.760958,-73.967245,member
1893442,A11D1F167EF1B8C6,classic_bike,2022-03-18 18:43:43,2022-03-18 18:53:41,W 56 St & 6 Ave,6809.07,E 58 St & 3 Ave,6762.02,40.763406,-73.977225,40.760958,-73.967245,member


In [17]:
csv_names[0]

'202201-citibike-tripdata.csv'

### Next up: Turn all station info into a separate smaller csv/table, remove NULL values, ...remove outliers?

In [ ]:
browser.quit()

### OLD Code with non-dynamic list names

In [6]:
data_2013to2016 = []
data_2017 = []
data_2018 = []
data_2019 = []
data_2020 = []
data_2021 = []
data_2022 = []


for i in range(0, (len(results) - 1 )): # the "-1" will filter out index.html
    date = results[i]["href"].split('/')[-1].split('-')[0]
    
    # filter out Jersey City (JC) data
    if (date != "JC"):
        year = int(date[0:4])
        if (year <= 2016):
            data_2013to2016.append(results[i]["href"])
        elif (year == 2017):
            data_2017.append(results[i]["href"])
        elif (year == 2018):
            data_2018.append(results[i]["href"])
        elif (year == 2019):
            data_2019.append(results[i]["href"])
        elif (year == 2020):
            data_2020.append(results[i]["href"])
        elif (year == 2021):
            data_2021.append(results[i]["href"])
        elif (year == 2022):
            data_2022.append(results[i]["href"])
           
            
data_2022

['https://s3.amazonaws.com/tripdata/202201-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/202202-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/202203-citibike-tripdata.csv.zip']

In [7]:
# Download zip files for a given year or year set
data_list = data_2022

limit = len(data_list)

In [8]:
# Create lists of zip names and csv names to use when calling the file
zip_names = []
csv_names = []

# for entry in results:
for i in range(0, limit):
    
    # Get the link
    link = data_list[i]

    # Use the link to get the zip and csv names, in case the .text is mismatched
    zip_name = link.split('/')[-1]
    zip_names.append(zip_name)

    csv_name = zip_name.split('.')[0] + ".csv"
    csv_names.append(csv_name)
   
    # Download the zip
    wget.download(link, out = "zipfiles") 
    
#     # Open and save as CSV
#     df = pd.read_csv(zf.open(csv_names[i]))
#     df.name = csv_names[i]
#     df.to_csv("CSVs")
    
    

100% [....................................................] 66434482 / 66434482

In [9]:
#Save as CSVs

for i in range(0, limit):
    
    # find and read the zip file
    find_zip = "zipfiles/" + zip_names[i]
    zf = zipfile.ZipFile(find_zip)
    df = pd.read_csv(zf.open(csv_names[i]))
    
    # name and create the csv
    df.name = csv_names[i]
    df.to_csv("data/" + csv_names[i])
    
    

C:\Users\emera\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
selected = 1

# find_zip = "zipfiles/" + zip_names[selected]
# zf = zipfile.ZipFile(find_zip)
df = pd.read_csv(zf.open(csv_names[selected]))

KeyError: "There is no item named '202202-citibike-tripdata.csv' in the archive"